In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

In [ ]:
import pandas as pd

# Đường dẫn đến file CSV
csv_file_path = '/content/data1.xlsx'

# Đọc dữ liệu từ file CSV và lưu vào DataFrame
data = pd.read_excel(csv_file_path)


In [ ]:


# Đọc tệp Excel với nhiều sheet
excel_file = pd.ExcelFile('/content/data1.xlsx')
dataframes = {}
# Lấy danh sách tên sheet trong tệp Excel
sheet_names = excel_file.sheet_names
for sheet_name in sheet_names:
    df = excel_file.parse(sheet_name)
    df.dropna(inplace=True)  # Loại bỏ bản ghi trống
    dataframes[sheet_name] = df
df_sheet1 = dataframes['data1']


In [ ]:
df_sheet1.columns

Index(['year', 'maCK', 'P/E', 'yeild', 'turnover TB', 'beta', 'abnormal_0',
       'concentration', 'Dividend', 'leverage', 'liquidity', 'profit on Sales',
       'EPS', 'EPS2', 'ROE', 'cash on sales', 'TATO', 'NPGR', 'growth',
       'news_sentiment', 'manipulation'],
      dtype='object')

In [ ]:
count = df_sheet1['manipulation'].value_counts()[1]

In [ ]:
count

20

In [ ]:
count = df_sheet1['manipulation'].value_counts()[-1]

In [ ]:
count

118

In [ ]:
X = df_sheet1.drop(columns=['year', 'maCK','manipulation'], axis=1)  # Drop the label column if necessary
# X = df.drop(columns=['year', 'maCK','manipulation'], axis=1)  # Drop the label column if necessary


In [ ]:
y = df_sheet1['manipulation']
# y = df['manipulation']


In [ ]:
count = y.shape[0]
count

138

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Chuẩn hóa Min-Max
min_max_scaler = MinMaxScaler()  # Tạo một đối tượng MinMaxScaler
X_train= min_max_scaler.fit_transform(X_train)
X_test= min_max_scaler.fit_transform(X_test)

In [ ]:
X_train

array([[0.06396486, 0.6010249 , 0.00083995, ..., 0.12721358, 0.18002991,
        0.        ],
       [0.05213594, 0.59625285, 0.07512092, ..., 0.14153014, 0.20024665,
        0.        ],
       [0.04743446, 0.56982873, 0.00485182, ..., 0.12406841, 0.14870765,
        0.        ],
       ...,
       [0.04004282, 0.60689081, 0.01958267, ..., 0.18088959, 0.14648934,
        0.125     ],
       [0.04022973, 0.56200237, 0.06864511, ..., 0.14227957, 0.18855569,
        0.        ],
       [0.06278076, 0.54136358, 0.00609713, ..., 0.16829408, 0.18867805,
        0.125     ]])

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
y_train

0     -1
1     -1
2     -1
3     -1
4     -1
      ..
181    1
182    1
183    1
184    1
185    1
Name: manipulation, Length: 186, dtype: int64

In [ ]:
svm_clf = SVC(random_state=42)
ann_clf = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',
                        learning_rate_init=0.01, max_iter=1000,random_state=42)
rf_clf = RandomForestClassifier(n_estimators=100,random_state=42)
lr_clf = LogisticRegression(random_state=42)

In [ ]:
# Khởi tạo tham số cho K-fold cross validation
n_splits = 5

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
# Vòng lặp chính
import numpy as np
val_preds = np.zeros((len(X_train), 4))
for i, (train_index, val_index) in enumerate(kf.split(X_train)):
    X_train_fold, y_train_fold = X_train[train_index], y_train[train_index]
    X_val_fold, y_val_fold = X_train[val_index], y_train[val_index]

    # Training các model trên fold hiện tại
    svm_clf.fit(X_train_fold, y_train_fold)
    ann_clf.fit(X_train_fold, y_train_fold)
    rf_clf.fit(X_train_fold, y_train_fold)
    lr_clf.fit(X_train_fold, y_train_fold)

    # Lưu trữ output của các model trên fold hiện tại
    val_preds[val_index, :] = np.column_stack((svm_clf.predict(X_val_fold),
                                                  ann_clf.predict(X_val_fold),
                                                  rf_clf.predict(X_val_fold),
                                                  lr_clf.predict(X_val_fold)))

In [ ]:
X_new= pd.DataFrame(val_preds)

In [ ]:
test_preds_l1= np.column_stack((svm_clf.predict(X_test),
                              ann_clf.predict(X_test),
                              rf_clf.predict(X_test),
                              lr_clf.predict(X_test)))

In [ ]:
ann_clf.fit(X_new, y_train)

MLPClassifier(learning_rate_init=0.01, max_iter=1000, random_state=42)

In [ ]:
y_pred = ann_clf.predict(test_preds_l1)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
print(f1_score(y_pred,y_test))
print(confusion_matrix(y_pred, y_test))

0.6666666666666666
[[24  1]
 [ 1  2]]
